In [1]:
import pandas as pd

confirmed_raw_url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv"
deaths_raw_url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Deaths.csv"
recovered_raw_url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Recovered.csv"

confirmed_df_raw = pd.read_csv(confirmed_raw_url)
deaths_df_raw = pd.read_csv(deaths_raw_url)
recovered_df_raw = pd.read_csv(recovered_raw_url)

In [2]:
latest_date = confirmed_df_raw.columns[-1]

confirmed_df_raw = confirmed_df_raw.drop(['Province/State', 'Lat', 'Long'], axis = 1) 
total_confirmed = confirmed_df_raw.groupby('Country/Region', as_index = False).agg(sum).sort_values(latest_date, ascending = False)
total_confirmed = pd.melt(total_confirmed, id_vars = 'Country/Region', var_name='Dates', value_name='Confirmed')
total_confirmed['Dates']= pd.to_datetime(total_confirmed['Dates']) 

deaths_df_raw = deaths_df_raw.drop(['Province/State', 'Lat', 'Long'], axis = 1) 
total_deaths = deaths_df_raw.groupby('Country/Region', as_index = False).agg(sum).sort_values(latest_date, ascending = False)
total_deaths = pd.melt(total_deaths, id_vars = 'Country/Region', var_name='Dates', value_name='Deaths')
total_deaths['Dates']= pd.to_datetime(total_deaths['Dates']) 

recovered_df_raw = recovered_df_raw.drop(['Province/State', 'Lat', 'Long'], axis = 1) 
total_recovered = recovered_df_raw.groupby('Country/Region', as_index = False).agg(sum).sort_values(latest_date, ascending = False)
total_recovered = pd.melt(total_recovered, id_vars = 'Country/Region', var_name='Dates', value_name='Recovered')
total_recovered['Dates']= pd.to_datetime(total_recovered['Dates']) 

In [3]:
from bokeh.models import ColumnDataSource

def get_country_datasource(_country):
    mask_confirmed = total_confirmed['Country/Region'] == _country
    mask_death = total_deaths['Country/Region'] == _country
    mask_recovered = total_recovered['Country/Region'] == _country
    
    _confirmed = total_confirmed[mask_confirmed].reset_index()
    _deaths = total_deaths[mask_death].reset_index()
    _recovered = total_recovered[mask_recovered].reset_index()
    
    _confirmed['Deaths'] = _deaths['Deaths']
    _confirmed['Recovered'] = _recovered['Recovered']

    _confirmed['Active'] = _confirmed['Confirmed'] - _confirmed['Deaths'] - _confirmed['Recovered']
    
    _confirmed = ColumnDataSource(_confirmed)
    
    return _confirmed

In [4]:
from bokeh.plotting import figure, output_file, show, save
from bokeh.models.tools import HoverTool
from ipykernel import kernelapp as app

country = "China"

confirmed = get_country_datasource(country)

output_file('line.html')
p = figure(title=country, x_axis_label='Date', y_axis_label='Count', 
           x_axis_type='datetime', plot_width=800, plot_height=500)
p.line(x='Dates', y='Confirmed', source=confirmed, color='black', line_width=3, legend="Confirmed Cases")
p.line(x='Dates', y='Deaths', source=confirmed, color='red', line_width=3, legend="Confirmed Deaths")
p.line(x='Dates', y='Recovered', source=confirmed, color='green', line_width=3, legend="Recoveries")

hover = HoverTool()
hover.tooltips=[('Confirmed', '@Confirmed'), ('Deaths', '@Deaths'), ('Recoveries', '@Recovered')]

p.add_tools(hover)
p.legend.location = 'top_left'

show(p)